# Kitchen Order Prep Time - Exploratory Data Analysis

This notebook performs comprehensive exploratory data analysis on kitchen order data to understand patterns in preparation times.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import Counter
from datetime import datetime

from config import Config
from src.data_loader import DataLoader

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

%matplotlib inline

## 1. Load and Validate Data

In [ ]:
# Load data
loader = DataLoader()
df_raw = loader.load_raw_data()
df = loader.validate_data(df_raw, require_target=True)
df = loader.parse_timestamps(df)
df = loader.clean_prep_times(df)

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nData types:\n{df.dtypes}")

In [ ]:
# Display first few rows
df.head(10)

In [ ]:
# Basic statistics
df.describe()

## 2. Data Quality Check

In [ ]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum())
print(f"\nMissing percentage:")
print((df.isnull().sum() / len(df) * 100).round(2))

In [ ]:
# Check unique values
print(f"Unique orders: {df['order_id'].nunique()}")
print(f"Unique kitchens: {df['kitchen_id'].nunique()}")
print(f"Import sources: {df['import_source'].unique()}")
print(f"\nOrders by import source:\n{df['import_source'].value_counts()}")

## 3. Target Variable Analysis: prep_time_seconds

In [ ]:
# Prep time statistics
prep_time_stats = df['prep_time_seconds'].describe()
print("Prep Time Statistics (seconds):")
print(prep_time_stats)
print(f"\nIn minutes:")
print(f"  Mean: {prep_time_stats['mean']/60:.2f}")
print(f"  Median: {prep_time_stats['50%']/60:.2f}")
print(f"  Std: {prep_time_stats['std']/60:.2f}")
print(f"  Min: {prep_time_stats['min']/60:.2f}")
print(f"  Max: {prep_time_stats['max']/60:.2f}")

In [ ]:
# Distribution of prep time
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Histogram
axes[0].hist(df['prep_time_seconds']/60, bins=50, edgecolor='black')
axes[0].set_xlabel('Prep Time (minutes)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Prep Time')
axes[0].axvline(df['prep_time_seconds'].mean()/60, color='red', linestyle='--', label='Mean')
axes[0].axvline(df['prep_time_seconds'].median()/60, color='green', linestyle='--', label='Median')
axes[0].legend()

# Box plot
axes[1].boxplot(df['prep_time_seconds']/60)
axes[1].set_ylabel('Prep Time (minutes)')
axes[1].set_title('Box Plot of Prep Time')

# Log-scale histogram
axes[2].hist(np.log1p(df['prep_time_seconds']), bins=50, edgecolor='black')
axes[2].set_xlabel('Log(Prep Time + 1)')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Log-scaled Distribution')

plt.tight_layout()
plt.show()

## 4. Temporal Analysis

In [ ]:
# Extract temporal features
df['hour'] = df['activated_at_local'].dt.hour
df['day_of_week'] = df['activated_at_local'].dt.dayofweek
df['date'] = df['activated_at_local'].dt.date
df['is_weekend'] = df['day_of_week'] >= 5

# Day names
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [ ]:
# Prep time by hour of day
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Average prep time by hour
hourly_prep = df.groupby('hour')['prep_time_seconds'].mean() / 60
axes[0].plot(hourly_prep.index, hourly_prep.values, marker='o', linewidth=2)
axes[0].set_xlabel('Hour of Day')
axes[0].set_ylabel('Average Prep Time (minutes)')
axes[0].set_title('Average Prep Time by Hour of Day')
axes[0].grid(True, alpha=0.3)
axes[0].set_xticks(range(24))

# Order count by hour
hourly_count = df['hour'].value_counts().sort_index()
axes[1].bar(hourly_count.index, hourly_count.values, color='skyblue', edgecolor='black')
axes[1].set_xlabel('Hour of Day')
axes[1].set_ylabel('Number of Orders')
axes[1].set_title('Order Volume by Hour of Day')
axes[1].set_xticks(range(24))

plt.tight_layout()
plt.show()

In [ ]:
# Prep time by day of week
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Box plot
df_plot = df.copy()
df_plot['day_name'] = df_plot['day_of_week'].map(lambda x: day_names[x])
day_order = [day_names[i] for i in range(7)]
sns.boxplot(data=df_plot, x='day_name', y='prep_time_seconds', order=day_order, ax=axes[0])
axes[0].set_xlabel('Day of Week')
axes[0].set_ylabel('Prep Time (seconds)')
axes[0].set_title('Prep Time Distribution by Day of Week')
axes[0].tick_params(axis='x', rotation=45)

# Average by day
daily_prep = df.groupby('day_of_week')['prep_time_seconds'].mean() / 60
axes[1].bar([day_names[i] for i in daily_prep.index], daily_prep.values, color='coral', edgecolor='black')
axes[1].set_xlabel('Day of Week')
axes[1].set_ylabel('Average Prep Time (minutes)')
axes[1].set_title('Average Prep Time by Day of Week')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Time series of average prep time
daily_avg = df.groupby('date')['prep_time_seconds'].agg(['mean', 'count']).reset_index()
daily_avg['mean_minutes'] = daily_avg['mean'] / 60

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].plot(daily_avg['date'], daily_avg['mean_minutes'], marker='o', linewidth=2)
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Average Prep Time (minutes)')
axes[0].set_title('Average Prep Time Over Time')
axes[0].grid(True, alpha=0.3)

axes[1].bar(daily_avg['date'], daily_avg['count'], color='lightgreen', edgecolor='black')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Number of Orders')
axes[1].set_title('Order Volume Over Time')

plt.tight_layout()
plt.show()

## 5. Kitchen Analysis

In [ ]:
# Kitchen statistics
kitchen_stats = df.groupby('kitchen_id').agg({
    'prep_time_seconds': ['mean', 'std', 'min', 'max', 'count']
}).round(2)
kitchen_stats.columns = ['mean', 'std', 'min', 'max', 'count']
kitchen_stats = kitchen_stats.sort_values('count', ascending=False)
print("Kitchen Statistics:")
print(kitchen_stats)

In [ ]:
# Prep time by kitchen
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Box plot
sns.boxplot(data=df, x='kitchen_id', y='prep_time_seconds', ax=axes[0])
axes[0].set_xlabel('Kitchen ID')
axes[0].set_ylabel('Prep Time (seconds)')
axes[0].set_title('Prep Time Distribution by Kitchen')
axes[0].tick_params(axis='x', rotation=45)

# Bar plot of averages
kitchen_avg = df.groupby('kitchen_id')['prep_time_seconds'].mean().sort_values(ascending=False) / 60
axes[1].bar(range(len(kitchen_avg)), kitchen_avg.values, color='purple', edgecolor='black')
axes[1].set_xticks(range(len(kitchen_avg)))
axes[1].set_xticklabels(kitchen_avg.index, rotation=45)
axes[1].set_xlabel('Kitchen ID')
axes[1].set_ylabel('Average Prep Time (minutes)')
axes[1].set_title('Average Prep Time by Kitchen')

plt.tight_layout()
plt.show()

## 6. Order Complexity Analysis

In [ ]:
# Parse item lists
def parse_items(items):
    if isinstance(items, str):
        try:
            return json.loads(items)
        except:
            return []
    return items if isinstance(items, list) else []

df['items_list'] = df['obfuscated_item_names'].apply(parse_items)
df['num_items'] = df['items_list'].apply(len)
df['unique_items'] = df['items_list'].apply(lambda x: len(set(x)))

print(f"Average number of items per order: {df['num_items'].mean():.2f}")
print(f"Max items in an order: {df['num_items'].max()}")
print(f"\nItems distribution:")
print(df['num_items'].value_counts().sort_index())

In [ ]:
# Prep time vs number of items
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot
axes[0].scatter(df['num_items'], df['prep_time_seconds']/60, alpha=0.5)
axes[0].set_xlabel('Number of Items')
axes[0].set_ylabel('Prep Time (minutes)')
axes[0].set_title('Prep Time vs Number of Items')
axes[0].grid(True, alpha=0.3)

# Box plot by number of items
sns.boxplot(data=df[df['num_items'] <= 10], x='num_items', y='prep_time_seconds', ax=axes[1])
axes[1].set_xlabel('Number of Items')
axes[1].set_ylabel('Prep Time (seconds)')
axes[1].set_title('Prep Time Distribution by Number of Items')

plt.tight_layout()
plt.show()

# Correlation
corr = df[['num_items', 'prep_time_seconds']].corr().iloc[0, 1]
print(f"Correlation between num_items and prep_time: {corr:.3f}")

## 7. Subtotal Analysis

In [ ]:
# Subtotal statistics
print("Subtotal Statistics:")
print(df['subtotal'].describe())

In [ ]:
# Prep time vs subtotal
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot
axes[0].scatter(df['subtotal'], df['prep_time_seconds']/60, alpha=0.5)
axes[0].set_xlabel('Subtotal ($)')
axes[0].set_ylabel('Prep Time (minutes)')
axes[0].set_title('Prep Time vs Subtotal')
axes[0].grid(True, alpha=0.3)

# Subtotal bins
df['subtotal_bin'] = pd.qcut(df['subtotal'], q=4, labels=['Low', 'Medium-Low', 'Medium-High', 'High'])
sns.boxplot(data=df, x='subtotal_bin', y='prep_time_seconds', ax=axes[1])
axes[1].set_xlabel('Subtotal Quartile')
axes[1].set_ylabel('Prep Time (seconds)')
axes[1].set_title('Prep Time by Subtotal Quartile')

plt.tight_layout()
plt.show()

# Correlation
corr = df[['subtotal', 'prep_time_seconds']].corr().iloc[0, 1]
print(f"Correlation between subtotal and prep_time: {corr:.3f}")

## 8. Import Source Analysis

In [ ]:
# Prep time by import source
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot
sns.boxplot(data=df, x='import_source', y='prep_time_seconds', ax=axes[0])
axes[0].set_xlabel('Import Source')
axes[0].set_ylabel('Prep Time (seconds)')
axes[0].set_title('Prep Time by Import Source')

# Average comparison
source_avg = df.groupby('import_source')['prep_time_seconds'].mean() / 60
axes[1].bar(source_avg.index, source_avg.values, color=['blue', 'green'], edgecolor='black')
axes[1].set_xlabel('Import Source')
axes[1].set_ylabel('Average Prep Time (minutes)')
axes[1].set_title('Average Prep Time by Import Source')

plt.tight_layout()
plt.show()

# Statistical test
from scipy import stats
web_prep = df[df['import_source'] == 'web']['prep_time_seconds']
app_prep = df[df['import_source'] == 'app']['prep_time_seconds']
t_stat, p_value = stats.ttest_ind(web_prep, app_prep)
print(f"\nt-test result: t={t_stat:.3f}, p={p_value:.4f}")
if p_value < 0.05:
    print("Significant difference in prep time between web and app orders")
else:
    print("No significant difference in prep time between web and app orders")

## 9. Item Analysis

In [ ]:
# Most common items
all_items = []
for items in df['items_list']:
    all_items.extend(items)

item_counts = Counter(all_items)
top_20_items = item_counts.most_common(20)

print("Top 20 Most Common Items:")
for i, (item, count) in enumerate(top_20_items, 1):
    print(f"{i:2d}. {item[:50]:<50} ({count} orders)")

In [ ]:
# Plot top items
plt.figure(figsize=(14, 8))
items, counts = zip(*top_20_items)
plt.barh(range(len(items)), counts, color='teal', edgecolor='black')
plt.yticks(range(len(items)), [item[:40] for item in items])
plt.xlabel('Number of Orders')
plt.title('Top 20 Most Ordered Items')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 10. Correlation Analysis

In [ ]:
# Correlation matrix
numerical_features = ['prep_time_seconds', 'subtotal', 'num_items', 'unique_items', 'hour', 'day_of_week']
corr_matrix = df[numerical_features].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Numerical Features')
plt.tight_layout()
plt.show()

## 11. Key Insights Summary

In [ ]:
print("="*80)
print("KEY INSIGHTS FROM EXPLORATORY DATA ANALYSIS")
print("="*80)
print(f"\n1. DATASET OVERVIEW")
print(f"   - Total orders: {len(df)}")
print(f"   - Unique kitchens: {df['kitchen_id'].nunique()}")
print(f"   - Date range: {df['date'].min()} to {df['date'].max()}")

print(f"\n2. PREP TIME DISTRIBUTION")
print(f"   - Mean: {df['prep_time_seconds'].mean()/60:.2f} minutes")
print(f"   - Median: {df['prep_time_seconds'].median()/60:.2f} minutes")
print(f"   - Std Dev: {df['prep_time_seconds'].std()/60:.2f} minutes")
print(f"   - Range: {df['prep_time_seconds'].min()/60:.2f} - {df['prep_time_seconds'].max()/60:.2f} minutes")

print(f"\n3. TEMPORAL PATTERNS")
peak_hour = df.groupby('hour')['prep_time_seconds'].mean().idxmax()
print(f"   - Peak prep time hour: {peak_hour}:00")
busiest_hour = df['hour'].value_counts().idxmax()
print(f"   - Busiest hour: {busiest_hour}:00 ({df['hour'].value_counts().max()} orders)")

print(f"\n4. KITCHEN DIFFERENCES")
kitchen_avg = df.groupby('kitchen_id')['prep_time_seconds'].mean() / 60
print(f"   - Fastest kitchen: {kitchen_avg.idxmin()} ({kitchen_avg.min():.2f} min avg)")
print(f"   - Slowest kitchen: {kitchen_avg.idxmax()} ({kitchen_avg.max():.2f} min avg)")

print(f"\n5. ORDER COMPLEXITY")
print(f"   - Avg items per order: {df['num_items'].mean():.2f}")
print(f"   - Correlation with prep time: {df[['num_items', 'prep_time_seconds']].corr().iloc[0,1]:.3f}")

print(f"\n6. SUBTOTAL IMPACT")
print(f"   - Avg subtotal: ${df['subtotal'].mean():.2f}")
print(f"   - Correlation with prep time: {df[['subtotal', 'prep_time_seconds']].corr().iloc[0,1]:.3f}")

print(f"\n7. IMPORT SOURCE")
for source in df['import_source'].unique():
    avg_time = df[df['import_source']==source]['prep_time_seconds'].mean() / 60
    count = (df['import_source']==source).sum()
    print(f"   - {source}: {avg_time:.2f} min avg ({count} orders)")

print("\n" + "="*80)
print("RECOMMENDATIONS FOR MODELING")
print("="*80)
print("1. Use time-based features (hour, day_of_week, rush_hour indicators)")
print("2. Kitchen ID is highly predictive - use target encoding")
print("3. Order complexity (num_items, subtotal) should be included")
print("4. Consider text features from item names (TF-IDF)")
print("5. Use gradient boosting models (XGBoost/LightGBM) for best results")
print("6. Implement time-based cross-validation to prevent leakage")
print("="*80)